# Imports

In [3]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import pandas as pd
import re
import numpy as np

# Data Collection

In [4]:
#creating a useragent
#parameters
header = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}

In [5]:
#H&M site catalog URL
url = 'https://www2.hm.com/en_us/men/products/jeans.html'
#Request to URL
page = requests.get(url, headers = header)
print(page)

# Beautiful soup object
soup = BeautifulSoup( page.text, 'html.parser' )

# ===================== Product Data ============================
products = soup.find( 'ul', class_='products-listing small' )
product_list = products.find_all( 'article', class_='hm-product-item')
                                 
# product id
product_id = [p.get( 'data-articlecode' ) for p in product_list]
                                 
# product category
product_category = [p.get( 'data-category' ) for p in product_list]
                                 
# product name
product_list = products.find_all( 'a', class_='link' )
product_name = [p.get_text() for p in product_list]
                                 
# price
product_list = products.find_all( 'span', class_='price regular' )
product_price = [p.get_text() for p in product_list]
data = pd.DataFrame( [product_id, product_category, product_name, product_price] ).T
                                 
data.columns = ['product_id', 'product_category', 'product_name', 'product_price']

<Response [200]>


In [6]:
data.head()

,product_id,product_category,product_name,product_price
0,1024256001,men_jeans_slim,Slim Jeans,$ 19.99
1,0985159001,men_jeans_skinny,Skinny Jeans,$ 24.99
2,0985159008,men_jeans_skinny,Skinny Jeans,$ 24.99
3,0690449043,men_jeans_ripped,Skinny Jeans,$ 39.99
4,0875105023,men_jeans_relaxed,Relaxed Jeans,$ 34.99


In [8]:
data.shape

(36, 4)

In [9]:

#empty dataframe
df_compositions = pd.DataFrame()

#unique columns for all products
aux = []

cols = ['Art. No.',
 'Composition',
 'Fit',
 'Product safety',
 'Size',
 'More sustainable materials']

df_pattern = pd.DataFrame(columns = cols)


for i in range(len(data)):  
    #API Requests
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'
    print('Product:{}'.format(i) +  '{}'.format(url))
    
    page = requests.get(url, headers = header)


    #BeautifulSoup object
    soup = BeautifulSoup(page.text, 'html.parser')

    product_list = soup.find_all('a', class_ = 'filter-option miniature active') + soup.find_all('a', class_ = 'filter-option miniature')
    color_item = [p.get('data-color') for p in product_list]

    #product id
    product_id = [p.get('data-articlecode') for p in product_list]

    #creating data frame with product id+color name
    df_color = pd.DataFrame((product_id, color_item)).T
    df_color.columns = ['product_id', 'color_name'] #renaming columns
    
    for j in range(len(df_color)):
        
        #API Requests
        url = 'https://www2.hm.com/en_us/productpage.' + df_color.loc[j, 'product_id'] + '.html'
        print('Color: {}'.format(url))
        page = requests.get(url, headers = header)

        #BeautifulSoup object
        soup = BeautifulSoup(page.text, 'html.parser')
        
        ########################  PRODUCT NAME  ###################################
        product_name = soup.find_all('h1', class_ = 'primary product-item-headline')
        product_name = product_name[0].get_text()
        
        ########################  PRODUCT PRICE  ###################################
        product_price = soup.find_all('div', class_ = 'primary-row product-item-price')
        product_price = re.findall(r'\d+\.?\d+', product_price[0].get_text())[0]
        
        #######################  COMPOSITION  ######################################
    
        product_composition_list = soup.find_all('div', class_ = 'pdp-description-list-item')
        product_composition = [list(filter(None, p.get_text().split("\n"))) for p in product_composition_list]

        #renaming labels
        df_composition = pd.DataFrame(product_composition).T
        df_composition.columns = df_composition.iloc[0]

        #deleting first row
        df_composition = df_composition.iloc[1:].fillna(method = 'ffill')

        #remove pocket lining, shell and lining
        df_composition['Composition'] = df_composition['Composition'].str.replace('Pocket: ', '', regex = True)
        df_composition['Composition'] = df_composition['Composition'].str.replace('Pocket lining: ', '', regex = True)
        df_composition['Composition'] = df_composition['Composition'].str.replace('Shell: ', '', regex = True)
        df_composition['Composition'] = df_composition['Composition'].str.replace('Lining: ', '', regex = True)
        
        #guarantee the same number of columns
        df_composition = pd.concat([df_pattern, df_composition], axis = 0)
        
        #rename columns
        df_composition.columns = ['product_id', 'composition', 'fit', 'product_safety', 'size', 'sustainable_materials']
        df_composition['product_name'] = product_name
        df_composition['product_price'] = product_price
            
        #keep new columns if they show up
        aux = aux + df_composition.columns.tolist() 

        #merge df_color and df_composition
        df_composition = pd.merge( df_composition, df_color, how = 'left', on = 'product_id')

        #all products
        df_compositions = pd.concat([df_compositions, df_composition], axis = 0)
        
# join showroom data + details
df_compositions['style_id'] = df_compositions['product_id'].apply(lambda x: x[:-3])
df_compositions['color_id'] = df_compositions['product_id'].apply(lambda x: x[-3:])

df_compositions['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

Product:0https://www2.hm.com/en_us/productpage.1024256001.html
Color: https://www2.hm.com/en_us/productpage.1024256001.html
Color: https://www2.hm.com/en_us/productpage.1024256002.html
Color: https://www2.hm.com/en_us/productpage.1024256003.html
Color: https://www2.hm.com/en_us/productpage.1024256004.html
Color: https://www2.hm.com/en_us/productpage.1024256005.html
Color: https://www2.hm.com/en_us/productpage.1024256006.html
Color: https://www2.hm.com/en_us/productpage.1024256007.html
Product:1https://www2.hm.com/en_us/productpage.0985159001.html
Color: https://www2.hm.com/en_us/productpage.0985159001.html
Color: https://www2.hm.com/en_us/productpage.0985159002.html
Color: https://www2.hm.com/en_us/productpage.0985159003.html
Color: https://www2.hm.com/en_us/productpage.0985159004.html
Color: https://www2.hm.com/en_us/productpage.0985159005.html
Color: https://www2.hm.com/en_us/productpage.0985159006.html
Color: https://www2.hm.com/en_us/productpage.0985159007.html
Color: https://www2.

Color: https://www2.hm.com/en_us/productpage.0690449028.html
Color: https://www2.hm.com/en_us/productpage.0690449035.html
Color: https://www2.hm.com/en_us/productpage.0690449036.html
Color: https://www2.hm.com/en_us/productpage.0690449040.html
Color: https://www2.hm.com/en_us/productpage.0690449043.html
Color: https://www2.hm.com/en_us/productpage.0690449046.html
Color: https://www2.hm.com/en_us/productpage.0690449051.html
Color: https://www2.hm.com/en_us/productpage.0690449056.html
Product:13https://www2.hm.com/en_us/productpage.0985159002.html
Color: https://www2.hm.com/en_us/productpage.0985159002.html
Color: https://www2.hm.com/en_us/productpage.0985159001.html
Color: https://www2.hm.com/en_us/productpage.0985159003.html
Color: https://www2.hm.com/en_us/productpage.0985159004.html
Color: https://www2.hm.com/en_us/productpage.0985159005.html
Color: https://www2.hm.com/en_us/productpage.0985159006.html
Color: https://www2.hm.com/en_us/productpage.0985159007.html
Color: https://www2.h

Product:25https://www2.hm.com/en_us/productpage.0875105024.html
Color: https://www2.hm.com/en_us/productpage.0875105024.html
Color: https://www2.hm.com/en_us/productpage.0875105001.html
Color: https://www2.hm.com/en_us/productpage.0875105002.html
Color: https://www2.hm.com/en_us/productpage.0875105003.html
Color: https://www2.hm.com/en_us/productpage.0875105009.html
Color: https://www2.hm.com/en_us/productpage.0875105011.html
Color: https://www2.hm.com/en_us/productpage.0875105015.html
Color: https://www2.hm.com/en_us/productpage.0875105016.html
Color: https://www2.hm.com/en_us/productpage.0875105017.html
Color: https://www2.hm.com/en_us/productpage.0875105018.html
Color: https://www2.hm.com/en_us/productpage.0875105023.html
Product:26https://www2.hm.com/en_us/productpage.1024256005.html
Color: https://www2.hm.com/en_us/productpage.1024256005.html
Color: https://www2.hm.com/en_us/productpage.1024256001.html
Color: https://www2.hm.com/en_us/productpage.1024256002.html
Color: https://www

In [10]:
df_compositions

,product_id,composition,fit,product_safety,size,sustainable_materials,product_name,product_price,color_name,style_id,color_id,scrapy_datetime
0,1024256001,"Polyester 65%, Cotton 35%",Slim fit,NaN,"The model is 185cm/6'1"" and wears a size 31/32",NaN,\n\t\t\t\t\t\t\t Slim Jeans,19.99,Black,1024256,001,2022-01-16 17:48:41
1,1024256001,"Cotton 99%, Spandex 1%",Slim fit,NaN,"The model is 185cm/6'1"" and wears a size 31/32",NaN,\n\t\t\t\t\t\t\t Slim Jeans,19.99,Black,1024256,001,2022-01-16 17:48:41
0,1024256002,"Cotton 99%, Spandex 1%",Slim fit,NaN,NaN,NaN,\n\t\t\t\t\t\t\t Slim Jeans,19.99,Light denim blue,1024256,002,2022-01-16 17:48:41
1,1024256002,"Polyester 65%, Cotton 35%",Slim fit,NaN,NaN,NaN,\n\t\t\t\t\t\t\t Slim Jeans,19.99,Light denim blue,1024256,002,2022-01-16 17:48:41
0,1024256003,"Cotton 99%, Spandex 1%",Slim fit,NaN,"The model is 189cm/6'2"" and wears a size 31/32",NaN,\n\t\t\t\t\t\t\t Slim Jeans,19.99,Light denim blue,1024256,003,2022-01-16 17:48:41
...,...,...,...,...,...,...,...,...,...,...,...,...
1,0974597002,Cotton 100%,Slim fit,NaN,NaN,Recycled cotton 20%,\n\t\t\t\t\t\t\t Slim Tapered Jeans,29.99,Denim blue,0974597,002,2022-01-16 17:48:41
0,0974597003,"Polyester 65%, Cotton 35%",Slim fit,NaN,NaN,Recycled cotton 21%,\n\t\t\t\t\t\t\t Slim Tapered Jeans,22.99,Black,0974597,003,2022-01-16 17:48:41
1,0974597003,"Cotton 98%, Spandex 2%",Slim fit,NaN,NaN,Recycled cotton 21%,\n\t\t\t\t\t\t\t Slim Tapered Jeans,22.99,Black,0974597,003,2022-01-16 17:48:41
0,0974597005,Cotton 100%,Slim fit,NaN,"The model is 187cm/6'2"" and wears a size 32/32",Recycled cotton 20%,\n\t\t\t\t\t\t\t Slim Tapered Jeans,22.99,Light denim blue,0974597,005,2022-01-16 17:48:41


# Data Cleaning

In [11]:
df_compositions.shape

(550, 12)

In [12]:
len(df_compositions['product_id'].unique())

114

In [13]:

#product id
#dropping NaN values
df_data = df_compositions.dropna(subset=['product_id'])

#product_name
df_data['product_name'] = df_data['product_name'].str.strip()
df_data['product_name'] = df_data['product_name'].str.replace(' ', '_').str.lower()
df_data['product_name'] = df_data['product_name'].str.replace('®', '').str.lower()


#product_price
df_data['product_price'] = df_data['product_price'].astype(float)

#color name
df_data['color_name'] = df_data['color_name'].str.replace(' ', '_').str.lower()

#fit
df_data['fit'] = df_data['fit'].apply(lambda x: x.replace(' ', '_').lower() if pd.notnull(x) else x)

#size number
df_data['size_number'] = df_data['size'].apply(lambda x: re.search('\d{3}', x).group(0) if pd.notnull(x) else x)
#df_data['size_number'] = df_data['size_number'].apply(lambda x: re.search('\d+', x).group(0) if pd.notnull(x) else x)

#size model
df_data['size_model'] = df_data['size'].str.extract('(\d+/\\d+)')
df_data['size_model'] = df_data['size_model'].apply(lambda x: x.replace('/', '_') if pd.notnull(x) else x)

#drop duplicated cells
#df_data = df_data.drop_duplicates()
#(subset=['product_id', 'product_category', 'product_name', 'product_price', 'scrapy_time', 'style_id', 'color_id', 'color_name', 'Fit', 'Composition', 'size_number', 'size_model'], keep='last' )

#reset index
df_data = df_data.reset_index(drop = True)

#break composition by comma
df1 = df_data['composition'].str.split(',', expand = True).reset_index(drop = True)


#creating new df with columns
# cotton | polyester | spandex | elasterell
df_ref = pd.DataFrame(index = np.arange(len(df_data)), columns = ['cotton', 'polyester', 'spandex', 'elasterell'])

############################### COMPOSITION #####################################

#cotton: 0, 1
#polyester: 0, 1
#spandex: 1, 2
#elasterell: 1

# -------------- cotton --------------
df_cotton_0 = df1.loc[df1[0].str.contains('Cotton', na = True), 0]
df_cotton_0.name = 'cotton'
df_cotton_1 = df1.loc[df1[1].str.contains('Cotton', na = True), 1]
df_cotton_1.name = 'cotton'

# combine cotton df's
df_cotton = df_cotton_0.combine_first(df_cotton_1)

df_ref = pd.concat([df_ref, df_cotton], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep = 'last')]


#------------ polyester ---------------
df_polyester_0 = df1.loc[df1[0].str.contains('Polyester', na = True), 0]
df_polyester_0.name = 'polyester'
df_polyester_1 = df1.loc[df1[1].str.contains('Polyester', na = True), 1]
df_polyester_1.name = 'polyester'

df_polyester = df_polyester_0.combine_first(df_polyester_1)
df_ref = pd.concat([df_ref, df_polyester], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep = 'last')]

#----------- spandex --------------------
df_spandex_1 = df1.loc[df1[1].str.contains('Spandex', na = True), 1]
df_spandex_1.name = 'spandex'
df_spandex_2 = df1.loc[df1[2].str.contains('Spandex', na = True), 2]
df_spandex_2.name = 'spandex'

#from jan 10 scrapy
#df_spandex_3 = df1.loc[df1[3].str.contains('Spandex', na = True), 3]
#df_spandex_3.name = 'spandex'

#df_spandex_c2 = df_spandex_1.combine_first(df_spandex_2)

df_spandex = df_spandex_1.combine_first(df_spandex_2)

df_ref = pd.concat([df_ref, df_spandex], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep = 'last')]


# ---------- elasterell --------------------
df_elasterell = df1.loc[df1[1].str.contains('Elasterell', na = True), 1]
df_elasterell.name = 'elasterell'

df_ref = pd.concat([df_ref, df_elasterell], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep = 'last')]

#combine join with product id 
df_aux = pd.concat([df_data['product_id'].reset_index(drop = True), df_ref], axis = 1)

#format composition data

df_aux['cotton']     = df_aux['cotton'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
df_aux['elasterell'] = df_aux['elasterell'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
df_aux['spandex']    = df_aux['spandex'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
df_aux['polyester']  = df_aux['polyester'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)

#final join
df_aux = df_aux.groupby('product_id').max().reset_index().fillna(0)
df_data = pd.merge(df_data, df_aux, on = 'product_id', how = 'left')

#drop columns
df_data = df_data.drop(columns = ['size', 'product_safety', 'sustainable_materials', 'composition'])

df_data = df_data.drop_duplicates()

In [20]:
df1[0].unique()

array(['Polyester 65%', 'Cotton 99%', 'Cotton 100%', 'Polyester 63%',
       'Cotton 98%', 'Polyester 100%', 'Cotton 77%', 'Cotton 78%',
       'Cotton 79%', 'Cotton 80%'], dtype=object)

In [14]:
df_data

,product_id,fit,product_name,product_price,color_name,style_id,color_id,scrapy_datetime,size_number,size_model,cotton,polyester,spandex,elasterell
0,1024256001,slim_fit,slim_jeans,19.99,black,1024256,001,2022-01-16 17:48:41,185,31_32,0.99,0.65,0.01,0.0
2,1024256002,slim_fit,slim_jeans,19.99,light_denim_blue,1024256,002,2022-01-16 17:48:41,NaN,NaN,0.99,0.65,0.01,0.0
4,1024256003,slim_fit,slim_jeans,19.99,light_denim_blue,1024256,003,2022-01-16 17:48:41,189,31_32,0.99,0.65,0.01,0.0
6,1024256004,slim_fit,slim_jeans,19.99,denim_blue,1024256,004,2022-01-16 17:48:41,NaN,NaN,0.99,0.65,0.01,0.0
8,1024256005,slim_fit,slim_jeans,19.99,dark_blue,1024256,005,2022-01-16 17:48:41,NaN,NaN,0.99,0.65,0.01,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,0974597006,slim_fit,slim_tapered_jeans,29.99,dark_gray,0974597,006,2022-01-16 17:48:41,NaN,NaN,1.00,0.00,0.02,0.0
542,0974597001,slim_fit,slim_tapered_jeans,29.99,denim_blue,0974597,001,2022-01-16 17:48:41,186,31_32,1.00,0.00,0.01,0.0
544,0974597002,slim_fit,slim_tapered_jeans,29.99,denim_blue,0974597,002,2022-01-16 17:48:41,NaN,NaN,1.00,0.00,0.01,0.0
546,0974597003,slim_fit,slim_tapered_jeans,22.99,black,0974597,003,2022-01-16 17:48:41,NaN,NaN,0.98,0.65,0.02,0.0


In [ ]:
df_data.to_csv(etl_hm.csv, encoding='utf-8', index=False)

In [55]:
df_ref

,cotton,polyester,spandex,elasterell
0,Cotton 35%,Polyester 65%,None,NaN
1,Cotton 99%,NaN,Spandex 1%,NaN
2,Cotton 99%,NaN,Spandex 1%,NaN
3,Cotton 35%,Polyester 65%,None,NaN
4,Cotton 99%,NaN,Spandex 1%,NaN
...,...,...,...,...
499,Cotton 100%,None,None,None
500,Cotton 35%,Polyester 65%,None,NaN
501,Cotton 98%,NaN,Spandex 2%,NaN
502,Cotton 100%,None,None,None


In [44]:
#cotton: 0, 1
#polyester: 0, 1
#spandex: 1, 2
#elasterell: 1



df1[0].unique()

array(['Polyester 65%', 'Cotton 99%', 'Cotton 100%', 'Polyester 63%',
       'Cotton 98%', 'Polyester 100%', 'Cotton 79%', 'Cotton 77%',
       'Cotton 78%', 'Cotton 80%', 'Cotton 90%', 'Polyester 80%',
       'Cotton 89%'], dtype=object)

In [45]:
df1[1].unique()

array([' Cotton 35%', ' Spandex 1%', None, ' Cotton 37%', ' Spandex 2%',
       ' Polyester 20%', ' Polyester 21%', ' Polyester 19%',
       ' Elasterell-P 8%', ' Cotton 20%', ' Elasterell-P 9%'],
      dtype=object)

In [46]:
df1[2].unique()

array([None, ' Spandex 1%', ' Spandex 2%'], dtype=object)

# Data Insert

In [22]:
import sqlite3
from sqlalchemy import create_engine

In [21]:
df_data.columns

Index(['product_id', 'fit', 'product_name', 'product_price', 'color_name',
       'style_id', 'color_id', 'scrapy_datetime', 'size_number', 'size_model',
       'cotton', 'polyester', 'spandex', 'elasterell'],
      dtype='object')

In [30]:
data_insert = df_data[[
    'product_id',
    'style_id',
    'color_id',
    'product_name',
    'color_name',
    'fit',
    'product_price',
    'size_number',
    'size_model',
    'cotton',
    'polyester',
    'spandex',
    'elasterell',
    'scrapy_datetime']]

In [25]:
data_insert.dtypes

product_id          object
style_id            object
color_id            object
product_name        object
color_name          object
fit                 object
product_price      float64
size_number         object
size_model          object
cotton             float64
polyester          float64
spandex            float64
elasterell         float64
scrapy_datetime     object
dtype: object

In [31]:
query_showroom_schema = '''
    CREATE TABLE vitrine(
    product_id         TEXT,
    style_id           TEXT,
    color_id           TEXT,
    product_name       TEXT,
    color_name         TEXT,
    fit                TEXT,
    product_price      REAL,
    size_number        TEXT,
    size_model         TEXT,
    cotton             REAL,
    polyester          REAL,
    spandex            REAL,
    elasterell         REAL,
    scrapy_datetime    TEXT
    
    
    
    )
'''

In [32]:
conn = sqlite3.connect('database_hm.sqlite')
cursor = conn.execute(query_showroom_schema)
conn.commit()

In [33]:
conn = create_engine('sqlite:///database_hm.sqlite')
data_insert.to_sql('vitrine', con = conn, if_exists = 'append', index = False)

In [34]:
query = '''
        SELECT * FROM vitrine'''

df = pd.read_sql_query(query, conn)
df.head()

,product_id,style_id,color_id,product_name,color_name,fit,product_price,size_number,size_model,cotton,polyester,spandex,elasterell,scrapy_datetime
0,1024256001,1024256,001,slim_jeans,black,slim_fit,19.99,185,31_32,0.99,0.65,0.01,0.0,2022-01-16 17:48:41
1,1024256002,1024256,002,slim_jeans,light_denim_blue,slim_fit,19.99,None,None,0.99,0.65,0.01,0.0,2022-01-16 17:48:41
2,1024256003,1024256,003,slim_jeans,light_denim_blue,slim_fit,19.99,189,31_32,0.99,0.65,0.01,0.0,2022-01-16 17:48:41
3,1024256004,1024256,004,slim_jeans,denim_blue,slim_fit,19.99,None,None,0.99,0.65,0.01,0.0,2022-01-16 17:48:41
4,1024256005,1024256,005,slim_jeans,dark_blue,slim_fit,19.99,None,None,0.99,0.65,0.01,0.0,2022-01-16 17:48:41
